In [4]:


import win32com.client


# Connect to Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6)
messages = inbox.Items

# Read emails and rewrite responses
for message in messages:
    subject = message.Subject
    body = message.Body
    print(subject)

RE: Question about ePAR 
RE: Information needed: Creative Teaching Cases e-book 
Re: AWS/Open AI/Twitter Nov-Dec Payments
IA Program Meeting on Wednesday 6 December from 11:30-12:30
RE: CISE Shirts pickup time
RE: CISE Shirts pickup time
Re: Twitter API, Open AI and AWS Summary 
RE: IA615
Christmas Day Potluck Party on Dec. 25, 2023, at 12:00 noon to 5:00 pm in Hartman or ZSH
IA Program Summit on Friday 12 January 2024
RE: Spring teaching time
Appointment Confirm
Christmas Day Potluck Party on 12/25/23, 12 noon to 5 pm in JMU Hartman Forum (Room 2021) 
Automatic reply: Appointment Confirm
Re: AWS/Open AI/Twitter Nov-Dec Payments
Re: AWS/Open AI/Twitter Nov-Dec Payments
RE: Appointment Confirm
RE: Appointment Confirm
RE: Appointment Confirm
Re: Holiday Gifts
Chrome River Expense Approval [Xuebin Wei]
Re: AWS/Open AI/Twitter Nov-Dec Payments
Re: AWS/Open AI/Twitter Nov-Dec Payments
FW: Information needed: Creative Teaching Cases e-book 
Re: Marry Christmas 
Re: H5P CoP structure
RE: H5P 